# スキャン画像の読み取り，位置合わせ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import glob
sys.path += [".."]
from src import file_io, align_images

In [ ]:
DIRNAME = "../data/raw_resized/"
SAVENAME = "../data/processed/"
metadata = {"marker_gaussian": (15, 3),
            "marker_ranges": ((0, 0, 200, 200), (0, -200, 200, None), (-200, -200, None, None), (-200, 0, None, 200))}
image_aligner = align_images.ImageAligner(metadata)
dataloader = file_io.read_images(DIRNAME)
p, img = next(dataloader)
image_aligner.fit(img)
for p, img in dataloader:
    new_img = image_aligner.transform_one(img)
    q = p.replace(DIRNAME, SAVENAME)
    file_io.save_image(q, new_img)





    

### マーカーの位置検出

In [ ]:
def find_markers(img, template, mask=None):
    if mask is not None:
        img = img * mask

    # テンプレートマッチング
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    
    sort_idx = np.argsort(res)[-3:]
    m_res = res[sort_idx]
    # m_res が何を返すかわからないのでこの先はやってみないとわからない
    # http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_template_matching/py_template_matching.html



### アフィン変換

In [ ]:
def align_image(base_markers, img_markers, img):
    h, w = img.shape
    M = cv2.getAffineTransform(img_markers, base_markers)
    new_img = cv2.warpAffine(img, M, (h, w), borderValue=255)  # 境界の外は白で埋める
    return new_img

### 画像の保存

In [ ]:
def save_image(path, img):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cv2.imwrite(path, img)

### 実行

In [ ]:
TEMP_PATH = ""
BASE_PATH = ""
DIRNAME = ""
SAVE_DIRNAME = ""

template = read_image(TEMP_PATH)
baseimg = read_image(BASE_PATH)
base_markers = find_markers(baseimg, template)
images = read_images(DIRNAME)
for p, img in images:
    markers = find_markers(img, templage)
    new_img = align_image(base_markers, markers, img)

    plt.imshow(new_img, cmap="gray")
    plt.show()

    # q = p.replace(DIRNAME, SAVE_DIRNAME)
    # save_image(q, new_img)